Code used to sanity-check data: plotting raw and written files, calculating various statistics

In [1]:
import os
import zarr
import dask.array as da
import matplotlib.pyplot as plt
from dask import compute
from dask.diagnostics import ProgressBar
import numpy as np
import xarray as xr

In [2]:
stored_jhf_hr_path = "/home/idies/workspace/turbulence-ceph-staging/sciserver-turbulence/stsabl2048high/stsabl2048high.zarr"
raw_jhf_hr_0_path = "/home/idies/workspace/turbulence-ceph-staging/ncar-jhf/hr/jhf.000.nc"
raw_jhf_hr_1_path = "/home/idies/workspace/turbulence-ceph-staging/ncar-jhf/hr/jhf.001.nc"
raw_jhf_hr_104_path = "/home/idies/workspace/turbulence-ceph-staging/ncar-jhf/hr/jhf.104.nc"

In [3]:
jhf_hr_zarr = zarr.open(stored_jhf_hr_path)
jhf_hr_netcdf_t0 = xr.open_dataset(raw_jhf_hr_0_path)
jhf_hr_netcdf_t1 = xr.open_dataset(raw_jhf_hr_1_path)
jhf_hr_netcdf_t104 = xr.open_dataset(raw_jhf_hr_104_path)

In [4]:
stored_jhf_lr_path = "/home/idies/workspace/turbulence-ceph-staging/sciserver-turbulence/stsabl2048low/stsabl2048low.zarr"
raw_jhf_lr_0_path = "/home/idies/workspace/turbulence-ceph-staging/ncar-jhf/lr/jhf.000.nc"
raw_jhf_lr_1_path = "/home/idies/workspace/turbulence-ceph-staging/ncar-jhf/lr/jhf.001.nc"
raw_jhf_lr_19_path = "/home/idies/workspace/turbulence-ceph-staging/ncar-jhf/lr/jhf.019.nc"


jhf_lr_zarr = zarr.open(stored_jhf_lr_path)
jhf_lr_netcdf_t0 = xr.open_dataset(raw_jhf_lr_0_path)
jhf_lr_netcdf_t1 = xr.open_dataset(raw_jhf_lr_1_path)
jhf_lr_netcdf_t19 = xr.open_dataset(raw_jhf_lr_19_path)

<font color="cyan">

# Remember, data is saved in `nnz-nny-nnx`
    
    
</font>

# Quick-Verify Correctness of data

1. Check if `data == 0`

2. Pick one $64^3$ chunk and compare it to raw NetCDF

### High-Rate

#### `zarr.info`

In [ ]:
jhf_hr_zarr

In [ ]:
jhf_hr_zarr['energy'].info

In [ ]:
jhf_hr_zarr['velocity'].info

In [ ]:
jhf_hr_zarr.info

#### Indexing, Compare to 0

In [ ]:
print("Checking whether field all zeros - True is bad!")

for t in range(105):
    print("t=", t, " - ", np.all(jhf_hr_zarr['temperature'][t,:64,:64,:64,0] == 0))

#### Comparing Real Values

In [ ]:
jhf_hr_zarr['temperature'][0,:10,0,0,0]

In [ ]:
jhf_lr_zarr['temperature'][0,:10,0,0,0]

In [ ]:
jhf_hr_zarr['temperature'][1,:10,0,0,0]

In [10]:
for t in range(93, 105):
    zarr_comparison_data = jhf_hr_zarr['temperature'][t,:64,:64,:64,0]

    raw_t_path = f"/home/idies/workspace/turbulence-ceph-staging/ncar-jhf/hr/jhf.{t:03d}.nc"
    raw_xr = xr.open_dataset(raw_t_path)
    raw_t = raw_xr['t'].isel(nnx=slice(0, 64), nny=slice(0, 64), nnz=slice(0, 64)).values
    
    print("t: ", t, np.all(zarr_comparison_data == raw_t))

t:  97 True
t:  98 True
t:  99 True
t:  100 True
t:  101 True
t:  102 True
t:  103 True
t:  104 True


In [ ]:
jhf_hr_zarr['temperature'][0,0,0,:10,0]

In [ ]:
jhf_hr_zarr['energy'][0,:10,0,0,0]

In [ ]:
np.array(jhf_hr_netcdf_t0['e'][:10,0,0])

Xarray complains about missing metadata. NOT Fixed by GPT o1

`written_ds_xr = xr.open_dataset(stored_jhf_path, engine='zarr', consolidated=False)`

In [ ]:
import numpy as np

np.all(ds['velocity'][0, :10,0,0, 0] == 0)

In [ ]:
import numpy as np

np.all(ds['velocity'][100, :10,0,0, 0] == 0)

In [ ]:
import numpy as np

np.all(ds['energy'][100, :10,0,0, 0] == 0)

### Low-Rate

#### Comparing Real Values

In [ ]:
jhf_lr_zarr['temperature'][0,:10,0,0,0]

In [ ]:
np.array(jhf_lr_netcdf_t0['t'][:10,0,0])

# Efficient Zarr full data Slice `z=0`

This can take a few minutes

 Sciserver doesn't allow localhost connections, so can't use Dask Cluster console Sciserver doesn't allow localhost connections, so can't use Dask Cluster console
 
<font color="green"> Lazy loading speeds up reading times 10x+</font>

## Function Definition

In [18]:
import os
import zarr
import dask.array as da
import matplotlib.pyplot as plt
from dask import compute
from dask.diagnostics import ProgressBar

plt.rcParams['image.cmap'] = 'inferno'

def process_zarr(stored_jhf_path, dataset):
    if dataset not in ['hr', 'lr']:
        raise ValueError("Dataset must be 'hr' or 'lr'.")

    # Open the Zarr group
    store = zarr.open_group(stored_jhf_path, mode='r')

    # Create Dask arrays for each variable
    data_arrays = {}
    variables = list(store.array_keys())
    for var_name in variables:
        zarr_array = store[var_name]
        dask_array = da.from_zarr(zarr_array)
        # Squeeze scalar variables so that shape becomes [time, Z, Y, X]
        if dask_array.shape[-1] == 1:
            dask_array = dask_array.squeeze(axis=-1)
        data_arrays[var_name] = dask_array

    # Function to collect data slices
    def collect_data_slices(data_arrays, variables, timesteps):
        data_slices = []
        titles = []
        for variable in variables:
            for timestep in timesteps:
                dask_array = data_arrays[variable]
                # Get slices along each dimension (assuming shape = [time, Z, Y, X] or similar)
                x_slice = dask_array[timestep, :, :, 0]
                y_slice = dask_array[timestep, :, 0, :]
                z_slice = dask_array[timestep, 0, :, :]
                
                # For vector variables, handle components separately
                if variable == 'velocity':
                    for component in range(3):
                        x_comp_slice = x_slice[..., component]
                        y_comp_slice = y_slice[..., component]
                        z_comp_slice = z_slice[..., component]
                        data_slices.extend([x_comp_slice, y_comp_slice, z_comp_slice])
                        titles.extend([
                            f"{variable} (component {component}) nnx=0 Timestep={timestep}",
                            f"{variable} (component {component}) nny=0 Timestep={timestep}",
                            f"{variable} (component {component}) nnz=0 Timestep={timestep}"
                        ])
                else:
                    data_slices.extend([x_slice, y_slice, z_slice])
                    titles.extend([
                        f"{variable} nnx=0 Timestep={timestep}",
                        f"{variable} nny=0 Timestep={timestep}",
                        f"{variable} nnz=0 Timestep={timestep}"
                    ])
        return data_slices, titles

    # Choose timesteps: here every 5th timestep until the end
    timesteps = range(0, data_arrays['energy'].shape[0], 5)  

    # Collect data slices for selected variables and timesteps
    data_slices, titles = collect_data_slices(
        data_arrays, 
        ['energy', 'temperature', 'pressure', 'velocity'], 
        timesteps
    )

    # Compute all data slices at once (lazy evaluation: one single compute call)
    with ProgressBar():
        computed_slices = compute(*data_slices)

    # Create a master folder for the dataset: zarr_slices/<dataset>
    master_folder = os.path.join("zarr_slices", dataset)
    os.makedirs(master_folder, exist_ok=True)

    # For each variable, create a subfolder inside the dataset folder
    variables_to_process = ['energy', 'temperature', 'pressure', 'velocity']
    for var in variables_to_process:
        var_folder = os.path.join(master_folder, var)
        os.makedirs(var_folder, exist_ok=True)

    # We'll have 3 slices per timestep for scalar variables,
    # and 9 slices per timestep for velocity (3 components × 3 slices).
    slices_per_timestep = {
        'energy': 3,
        'temperature': 3,
        'pressure': 3,
        'velocity': 9
    }

    # Loop over each variable and each timestep to save the plots
    idx = 0
    for variable in variables_to_process:
        var_folder = os.path.join(master_folder, variable)
        for timestep in timesteps:
            subset = computed_slices[idx : idx + slices_per_timestep[variable]]
            subset_titles = titles[idx : idx + slices_per_timestep[variable]]
            for data_slice, title in zip(subset, subset_titles):
                plt.figure()
                plt.imshow(data_slice)
                plt.title(title)
                plt.gca().invert_yaxis()
                plt.colorbar()
                
                # Create a filename for saving in the variable subfolder
                safe_title = title.replace(" ", "_").replace("=", "_")
                filename = os.path.join(var_folder, f"Timestep_{timestep}_{safe_title}.png")
                plt.savefig(filename, dpi=150, bbox_inches='tight')
                plt.close()
            idx += slices_per_timestep[variable]

## Function Calls

In [19]:
process_zarr(stored_jhf_lr_path, "lr")

[########################################] | 100% Completed | 45.81 s


In [20]:
process_zarr(stored_jhf_hr_path, "hr")

[##########                              ] | 25% Completed | 195.81 s



KeyboardInterrupt



# Combined Mean Temp and Slice Plotting

## Function Definition

In [21]:
import os
import dask.array as da
from dask import compute
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt
import numpy as np

def plot_slices_and_mean_z(
    ds_zarr_group, 
    variables, 
    time_start=0, 
    time_stop=105, 
    time_step=5, 
    z_step=128,
    master_slice_folder="zarr_slices",
    mean_output_folder="mean_temperature_plots"
):
    """
    1) Creates a Dask array for each variable from the Zarr group ds_zarr_group.
    2) Gathers slice arrays (X=0, Y=0, and Z=range(0, z_dim, z_step)) for each variable/time.
    3) Optionally gathers the mean-temp-across-Z (every z_step along Z).
    4) Performs exactly one .compute() to load everything.
    5) Plots and/or saves images if the respective output folder is provided.

    Parameters
    ----------
    ds_zarr_group : zarr.hierarchy.Group
        A Zarr group, e.g. from zarr.open_group(...), with arrays named by `variables`.
        Each array shape typically [time, z, y, x, 1 or 3].
    variables : list of str
        E.g. ["energy", "temperature", "pressure", "velocity"]
    time_start, time_stop, time_step : int
        We will process times in range(time_start, time_stop, time_step).
        Make sure time_stop <= ds['temperature'].shape[0] (if you use 'temperature').
    z_step : int
        Step size along Z dimension for the z-slice plotting,
        and also for the mean calculation if mean_output_folder is not None.
    master_slice_folder : str or None
        Where to save slice images. If None, skip slice plotting & saving altogether.
    mean_output_folder : str or None
        Where to save mean-temperature plots. If None, skip mean-temperature calculation/plotting.
    """

    # -------------------------------------------------------------------------
    # 1) Convert each Zarr array to a Dask array (lazy)
    # -------------------------------------------------------------------------
    data_arrays = {}
    for var in variables:
        zarr_array = ds_zarr_group[var]  # shape e.g. (105, 2048, 2048, 2048, 1)
        dask_array = da.from_zarr(zarr_array)  # now a Dask array, fully lazy
        # If last dim == 1, squeeze it out => shape => (time, z, y, x)
        if dask_array.shape[-1] == 1:
            dask_array = dask_array.squeeze(axis=-1)
        data_arrays[var] = dask_array

    # -------------------------------------------------------------------------
    # 2) Build slices for time and for z
    # -------------------------------------------------------------------------
    # We'll do time slicing using slice(time_start, time_stop, time_step).
    # e.g. if time_start=0, time_stop=105, time_step=5 => times = [0,5,10,...,100]
    time_slice = slice(time_start, time_stop, time_step)
    
    # For the z dimension in the "z-slice" plotting, we want every z_step
    # from z=0..(z_dim). So we do z_slice = slice(None, None, z_step)
    z_slice = slice(None, None, z_step)

    # -------------------------------------------------------------------------
    # 3a) Gather slice arrays: X=0, Y=0, Z=range(0, z_dim, z_step)
    # -------------------------------------------------------------------------
    slice_arrays = []   # each item will be a Dask array
    slice_titles = []   # matching titles

    for var in variables:
        arr = data_arrays[var]  
        # shape => (time, z, y, x) or (time, z, y, x, 3) if velocity was not squeezed

        if (var == "velocity") and (arr.ndim == 5):
            # Velocity shape => (time, z, y, x, 3)
            for comp in range(3):
                # X=0 => arr[time_slice, :, :, 0, comp]
                vx0 = arr[time_slice, :, :, 0, comp]
                slice_arrays.append(vx0)
                slice_titles.append(f"{var} (comp {comp}) x=0")

                # Y=0 => arr[time_slice, :, 0, :, comp]
                vy0 = arr[time_slice, :, 0, :, comp]
                slice_arrays.append(vy0)
                slice_titles.append(f"{var} (comp {comp}) y=0")

                # Z= every z_step => arr[time_slice, ::z_step, :, :, comp]
                vzAll = arr[time_slice, z_slice, :, :, comp]
                slice_arrays.append(vzAll)
                slice_titles.append(f"{var} (comp {comp}) z=all(step={z_step})")

        else:
            # Scalar variable shape => (time, z, y, x)
            # X=0
            x0 = arr[time_slice, :, :, 0]
            slice_arrays.append(x0)
            slice_titles.append(f"{var} x=0")

            # Y=0
            y0 = arr[time_slice, :, 0, :]
            slice_arrays.append(y0)
            slice_titles.append(f"{var} y=0")

            # Z= every z_step => arr[time_slice, ::z_step, :, :]
            zAll = arr[time_slice, z_slice, :, :]
            slice_arrays.append(zAll)
            slice_titles.append(f"{var} z=all(step={z_step})")

    # -------------------------------------------------------------------------
    # 3b) Gather the Dask array for "mean temperature across Z" (optional)
    # -------------------------------------------------------------------------
    mean_temp_dask = None
    if (mean_output_folder is not None) and ("temperature" in data_arrays):
        temp_arr = data_arrays["temperature"]  # shape => (time, z, y, x)
        stepped = temp_arr[time_slice, z_slice, :, :]  # [time, #z_slices, y, x]
        # Mean over y,x => shape => [num_times, #z_slices]
        mean_temp_dask = stepped.mean(axis=(2, 3))

    # -------------------------------------------------------------------------
    # 4) Single .compute() for all
    # -------------------------------------------------------------------------
    to_compute = slice_arrays[:]
    if mean_temp_dask is not None:
        to_compute.append(mean_temp_dask)

    if not to_compute:
        print("No data to load or compute. Exiting.")
        return

    print("Building Dask graph (lazy). Now calling .compute() with a ProgressBar...")
    with ProgressBar():
        results = compute(*to_compute)
    slice_results = results[:len(slice_arrays)]
    mean_results = results[-1] if mean_temp_dask is not None else None

    # Build the list of timesteps actually used
    times = list(range(time_start, time_stop, time_step))

    # -------------------------------------------------------------------------
    # 5) Plot & Save the slice results (if master_slice_folder is not None)
    # -------------------------------------------------------------------------
    if master_slice_folder is not None:
        os.makedirs(master_slice_folder, exist_ok=True)
        print("Plotting slice images to:", master_slice_folder)

        for slice_result, title in zip(slice_results, slice_titles):
            # slice_result shape can be:
            #   [num_times, z, y]   if x=0
            #   [num_times, z, x]   if y=0
            #   [num_times, #z_slices, y, x] if it's the z= slice
            #
            # We'll do a for-loop over times, then if there's an extra dimension
            # for #z_slices, we do another nested loop.

            for i, t in enumerate(times):
                data_for_this_time = slice_result[i] 
                # data_for_this_time shape => [z,y], [z,x], or [#z_slices, y, x], etc.

                if data_for_this_time.ndim == 2:
                    # Single 2D slice: Just plot once
                    plt.figure()
                    plt.imshow(data_for_this_time)
                    plt.gca().invert_yaxis()
                    plt.colorbar()
                    plt.title(f"{title}, t={t}")

                    safe_title = title.replace(" ", "_").replace("=", "_")
                    fname = os.path.join(
                        master_slice_folder, 
                        f"{safe_title}_t_{t}.png"
                    )
                    plt.savefig(fname, dpi=150, bbox_inches='tight')
                    plt.close()

                elif data_for_this_time.ndim == 3:
                    # shape => [numZ, y, x], so we have multiple slices
                    numZ = data_for_this_time.shape[0]
                    for z_i in range(numZ):
                        single_slice = data_for_this_time[z_i]  # shape => [y, x]

                        plt.figure()
                        plt.imshow(single_slice)
                        plt.gca().invert_yaxis()
                        plt.colorbar()
                        plt.title(f"{title}, t={t}, zstep_idx={z_i}")

                        safe_title = title.replace(" ", "_").replace("=", "_")
                        fname = os.path.join(
                            master_slice_folder, 
                            f"{safe_title}_t_{t}_zidx_{z_i}.png"
                        )
                        plt.savefig(fname, dpi=150, bbox_inches='tight')
                        plt.close()
                else:
                    print(f"Warning: Unexpected shape {data_for_this_time.shape} for {title}, skipping plot.")

        print("Done plotting slices!")
    else:
        print("master_slice_folder=None => Skipping slice plotting.")

    # -------------------------------------------------------------------------
    # 6) Plot the mean temperature results (if mean_output_folder and mean_results)
    # -------------------------------------------------------------------------
    if mean_results is not None:
        os.makedirs(mean_output_folder, exist_ok=True)
        print("Plotting mean temperature to:", mean_output_folder)

        # mean_results shape => [num_times, #z_slices]
        # The z_slices are from z=0..(some max).. step z_step
        z_indices_list = list(range(0, data_arrays["temperature"].shape[1], z_step))

        for i, t in enumerate(times):
            row = mean_results[i]  # shape => [#z_slices]

            plt.figure()
            plt.plot(z_indices_list, row)
            plt.title(f"Mean Temperature across Z (t={t})")
            plt.xlabel("Z index")
            plt.ylabel("Avg Temp")
            
            fname = os.path.join(mean_output_folder, f"mean_z_temperature_t_{t}.png")
            plt.savefig(fname, dpi=150, bbox_inches='tight')
            plt.close()
            print(f"Saved plot as {fname}")
    else:
        if mean_output_folder is None:
            print("mean_output_folder=None => Skipping mean temperature calculation and plots.")
        else:
            print("No temperature array or empty results => skipping mean temperature.")
    
    print("All done.")

## Zarr

### High Rate

- [ ] TODO this needs editing to add HR plots to HR folder, LR plots to own folder

#### Across Z

In [ ]:
variables = ["energy", "temperature", "pressure", "velocity"]

plot_slices_and_mean_z(
    ds_zarr_group=jhf_hr_zarr,
    variables=variables,
    time_start=0,
    time_stop=105,  # up to but not including 105 => indices 0..104
    time_step=5,
    z_step=16,
    master_slice_folder="zarr_slices",
    mean_output_folder="mean_temperature_plots"
)

Building Dask graph (lazy). Now calling .compute() with a ProgressBar...


### Low-Rate

In [ ]:
variables = ["energy", "temperature", "pressure", "velocity"]

plot_slices_and_mean_z(
    ds_zarr_group=jhf_lr_zarr,
    variables=variables,
    time_start=0,
    time_stop=20,  # up to but not including 105 => indices 0..104
    time_step=2,
    z_step=16,
    master_slice_folder="zarr_slices",
    mean_output_folder="mean_temperature_plots"
)

## Across X

- [ ] TODO if necessary

## Original NetCDF

In [ ]:
plot_mean_temp_across_z(data_arrays, timesteps=range(0, 106, 5), data_type="original", spacing=128)

# Compare Zarr to NetCDF correctness

## Function Definitions

In [14]:
import os
import numpy as np
import xarray as xr
import dask.array as da
from dask.diagnostics import ProgressBar
from dask import compute
import xarray as xr

def compare_zarr_and_netcdf_allvars(
    zarr_group,
    netcdf_path_pattern,
    times=range(0, 105),  # valid time indices
    z_slice=64,
    y_slice=64,
    x_slice=64,
    netcdf_chunk=64
):
    """
    Compare all JHF variables between a Zarr group and multiple NetCDF files 
    for the given timesteps, using lazy Dask operations in one pass.

    Parameters
    ----------
    zarr_group : zarr.hierarchy.Group
        Zarr group containing the 4 JHF variables:
        ["temperature", "pressure", "energy", "velocity"].
        Each variable is shape [time, z, y, x, (1 or 3)].
    netcdf_path_pattern : str
        File path pattern for NetCDF files. 
        E.g. "/path/to/jhf.*.nc" if each file is one timestep.
    times : iterable
        Timestep indices (e.g. range(0,105)) to compare in one go.
    z_slice, y_slice, x_slice : int
        Compare subregion [0:z_slice, 0:y_slice, 0:x_slice].
        Typically something small for quick checks (64, 128, etc.).
    netcdf_chunk : int
        Chunk size to use for opening NetCDF 
        (e.g. 64 => chunks={"nnz": 64, "nny": 64, "nnx": 64}).
    """

    #---------------------
    # 1) OPEN NETCDF
    #---------------------
    ds_nc = xr.open_mfdataset(
        netcdf_path_pattern,
        combine="nested",
        concat_dim="time",
        parallel=True,
        chunks={"nnz": netcdf_chunk, "nny": netcdf_chunk, "nnx": netcdf_chunk}
    )
    # The NetCDF variables we expect: "t", "p", "e", "u", "v", "w"
    # Each file is presumably one time step, stacked along "time".

    #---------------------
    # 2) PREP Zarr as Dask arrays
    #---------------------
    # We'll create xarray or Dask arrays from the Zarr group.
    # That ensures everything is truly lazy until .compute().
    var_map = {}
    for var in ["temperature", "pressure", "energy", "velocity"]:
        zarr_array = zarr_group[var]  # e.g. shape [time, z, y, x, 3 or 1]
        # Convert to a Dask array:
        dask_arr = da.from_zarr(zarr_array)
        # If last dim == 1, squeeze it out so shape becomes [time, z, y, x]
        if dask_arr.shape[-1] == 1:
            dask_arr = dask_arr.squeeze(axis=-1)
        # Wrap it in an xarray.DataArray for easier .isel():
        var_map[var] = xr.DataArray(
            dask_arr,
            dims=("time", "z", "y", "x") if var != "velocity" else ("time", "z", "y", "x", "component")
        )

    #---------------------
    # 3) BUILD COMPARISON
    #---------------------
    # We'll compare these variable pairs:
    #   Zarr => NetCDF
    #   "temperature" => "t"
    #   "pressure"    => "p"
    #   "energy"      => "e"
    #   "velocity"[..., comp=0] => "u"
    #   "velocity"[..., comp=1] => "v"
    #   "velocity"[..., comp=2] => "w"
    # We'll gather all eq_arrays in a list, then do one single compute.
    eq_arrays = []
    eq_descriptions = []

    # Helper to slice netcdf
    def slice_nc(var_name):
        return ds_nc[var_name].isel(
            time=times, 
            nnz=slice(0, z_slice), 
            nny=slice(0, y_slice), 
            nnx=slice(0, x_slice)
        )
    # Helper to slice zarr
    def slice_zarr(var_name):
        # if "velocity", we'll handle it separately below
        return var_map[var_name].isel(
            time=slice(min(times), max(times)+1),  # a bit broader, then we'll index in a second step
            z=slice(0, z_slice),
            y=slice(0, y_slice),
            x=slice(0, x_slice),
        ).isel(time=times if isinstance(times, list) else range(len(times)))
        # Explanation: if `times` is e.g. range(0,105,5), xarray can handle slice. 
        # If you pass a range object directly, xarray handles it. 
        # Or we cast to a list to ensure it can index properly.

    # Compare temperature => t
    if "temperature" in var_map and "t" in ds_nc.variables:
        zarr_temp = slice_zarr("temperature")
        nc_temp   = slice_nc("t")
        eq_temp   = (zarr_temp == nc_temp)
        eq_arrays.append(eq_temp) 
        eq_descriptions.append("temperature vs t")

    # Compare pressure => p
    if "pressure" in var_map and "p" in ds_nc.variables:
        zarr_pres = slice_zarr("pressure")
        nc_pres   = slice_nc("p")
        eq_pres   = (zarr_pres == nc_pres)
        eq_arrays.append(eq_pres)
        eq_descriptions.append("pressure vs p")

    # Compare energy => e
    if "energy" in var_map and "e" in ds_nc.variables:
        zarr_ener = slice_zarr("energy")
        nc_ener   = slice_nc("e")
        eq_ener   = (zarr_ener == nc_ener)
        eq_arrays.append(eq_ener)
        eq_descriptions.append("energy vs e")

    # Compare velocity => (u, v, w)
    if "velocity" in var_map and all(comp in ds_nc.variables for comp in ["u", "v", "w"]):
        # shape => [time, z, y, x, component=3]
        zarr_vel = var_map["velocity"].isel(
            time=slice(min(times), max(times)+1),
            z=slice(0, z_slice),
            y=slice(0, y_slice),
            x=slice(0, x_slice),
        ).isel(time=times if isinstance(times, list) else range(len(times)))

        for comp_idx, comp_name in enumerate(["u", "v", "w"]):
            nc_comp = slice_nc(comp_name)  # shape [time, nnz, nny, nnx]
            zarr_comp = zarr_vel.isel(component=comp_idx)  # shape [time, z, y, x]
            eq_comp = (zarr_comp == nc_comp)
            eq_arrays.append(eq_comp)
            eq_descriptions.append(f"velocity[{comp_idx}] vs {comp_name}")

    # If eq_arrays is empty, no matching variables found
    if not eq_arrays:
        print("No matching variables to compare!")
        ds_nc.close()
        return

    #---------------------
    # 4) SINGLE COMPUTE
    #---------------------
    print(f"Built {len(eq_arrays)} comparisons. Now computing with Dask (lazy).")
    with ProgressBar():
        eq_results = compute(*eq_arrays)
        # eq_results is a tuple of bool xarray.DataArray objects.

    #---------------------
    # 5) REPORT PER TIMESTEP
    #---------------------
    # Each eq_result => shape [time, z_slice, y_slice, x_slice], boolean
    # We'll do a .all(dim=["z","y","x"]) => shape [time], for each eq_result.
    # Then print out each time’s True/False. 
    # If you used different dim names, adapt accordingly.

    # Convert `times` to a list to iterate easily
    times_list = list(times)  # e.g. [0,1,2,...,104] or some subset
    for eq_array, desc in zip(eq_results, eq_descriptions):
        # eq_array is an xarray.DataArray of shape [time, z_slice, y_slice, x_slice]
        # Do a .all over spatial dims
        # If dims are named ("time", "z", "y", "x"), we do eq_array.all(dim=["z","y","x"])
        # If they’re named differently, adapt.
        eq_per_t = eq_array.all(dim=("z","y","x"))

        print(f"\n=== {desc} ===")
        for i, tval in enumerate(times_list):
            is_match = bool(eq_per_t[i].values)  # True/False
            print(f"  t={tval}, match={is_match}")

    ds_nc.close()
    print("Comparison done. All NetCDF files closed.")

## High-Rate

In [ ]:
# 2) Provide the NetCDF path pattern
nc_pattern = "/home/idies/workspace/turbulence-ceph-staging/ncar-jhf/hr/jhf.*.nc"  
# (Assumes jhf.000.nc, jhf.001.nc, ... each containing variables t, p, e, u, v, w)

# 3) Choose timesteps and slice sizes
times_to_compare = range(0, 105, 5)  # e.g. [0, 5, 10, 15, ..., 100]
z_slice_size = 64
y_slice_size = 64
x_slice_size = 64

compare_zarr_and_netcdf_allvars(
    zarr_group=jhf_hr_zarr,
    netcdf_path_pattern=nc_pattern,
    times=times_to_compare,
    z_slice=z_slice_size,
    y_slice=y_slice_size,
    x_slice=x_slice_size,
    netcdf_chunk=64  # netCDF chunk in nnz, nny, nnx
)

## Low-Rate

In [ ]:
# 2) Provide the NetCDF path pattern
nc_pattern = "/home/idies/workspace/turbulence-ceph-staging/ncar-jhf/lr/jhf.*.nc"  
# (Assumes jhf.000.nc, jhf.001.nc, ... each containing variables t, p, e, u, v, w)

# 3) Choose timesteps and slice sizes
times_to_compare = range(0, 20, 5)  # e.g. [0, 5, 10, 15, ..., 100]
z_slice_size = 64
y_slice_size = 64
x_slice_size = 64

# 4) Call the function
compare_zarr_and_netcdf_allvars(
    zarr_group=jhf_hr_zarr,
    netcdf_path_pattern=nc_pattern,
    times=times_to_compare,
    z_slice=z_slice_size,
    y_slice=y_slice_size,
    x_slice=x_slice_size,
    netcdf_chunk=64  # netCDF chunk in nnz, nny, nnx
)